##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 펭귄 데이터세트를 사용하는 간단한 TFX 파이프라인 튜토리얼

***간단한 TFX 파이프라인을 실행하는 짧은 튜토리얼입니다.***

참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행"을 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/penguin_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table></div>

이 노트북 기반 튜토리얼에서는 간단한 분류 모델을 위한 TFX 파이프라인을 생성하고 실행합니다. 파이프라인은 세 가지 필수 TFX 구성 요소인 ExampleGen, Trainer 및 Pusher로 구성되며 데이터 가져오기, 모델 훈련 및 훈련한 모델 내보내기와 같은 가장 최소한의 ML 워크플로를 포함합니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.

## 설정하기

먼저 TFX Python 패키지를 설치하고 모델에 사용할 데이터세트를 다운로드해야 합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인해야 합니다. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치


In [ ]:
!pip install -U tfx

### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 런타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

TensorFlow 및 TFX 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 변수 설정하기

파이프라인을 정의하는 데 사용되는 변수가 몇 가지 있습니다. 이러한 변수를 원하는 대로 사용자 정의할 수 있습니다. 기본적으로 파이프라인의 모든 출력은 현재 디렉터리 아래에 생성됩니다.

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 예제 데이터 준비하기

TFX 파이프라인에서 사용할 예제 데이터세트를 다운로드합니다. 사용하는 데이터세트는 다른 [TFX 예제](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)에서도 사용하는 [Palmer Penguins 데이터세트](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)입니다.

이 데이터세트에는 4가지 숫자 특성이 있습니다.

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

모든 특성이 이미 [0,1] 범위를 갖도록 정규화되었습니다. 펭귄의 `species`를 예측하는 분류 모델을 빌드합니다.

TFX ExampleGen은 디렉터리로부터 입력을 읽기 때문에 디렉터리를 생성한 후 데이터세트를 디렉터리에 복사해야 합니다.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

CSV 파일을 간단히 살펴보세요.

In [ ]:
!head {_data_filepath}

5가지 값을 볼 수 있어야 합니다. `species`는 0이나 1이나 2 중 하나이며 다른 모든 특성은 0과 1 사이의 값을 가져야 합니다.

## 파이프라인 생성하기

TFX 파이프라인은 Python API를 사용하여 정의됩니다. 다음 3가지 구성 요소로 구성된 파이프라인을 정의합니다.

- CsvExampleGen: 데이터 파일을 읽고 읽은 데이터 파일을 추가 처리를 위해 TFX 내부 형식으로 변환합니다. 다양한 형식에 대한 여러 [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen)이 있습니다. 이 튜토리얼에서는 CSV 파일 입력을 취하는 CsvExampleGen을 사용합니다.
- Trainer: ML 모델을 훈련합니다. [Trainer 구성 요소](https://www.tensorflow.org/tfx/guide/trainer)는 사용자로부터의 모델 정의 코드를 필요로 합니다. TensorFlow API를 사용하여 모델을 훈련하고 *saved_model* 형식으로 저장하는 방법을 지정할 수 있습니다.
- Pusher: TFX 파이프라인 외부의 훈련된 모델을 복사합니다. [Pusher 구성 요소](https://www.tensorflow.org/tfx/guide/pusher)는 훈련한 ML 모델의 배포 프로세스로 생각할 수 있습니다.

파이프라인을 실제로 정의하기 전에 먼저 Trainer 구성 요소에 대한 모델 코드를 작성해야 합니다.

### 모델 훈련 코드 작성하기

TensorFlow Keras API를 사용하여 분류를 위한 간단한 DNN 모델을 생성합니다. 이 모델 훈련 코드는 별도의 파일에 저장됩니다.

이 튜토리얼에서는 Keras 기반 모델을 지원하는 TFX의 [Generic Trainer](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer)를 사용합니다. 이를 위해 `Trainer` 구성 요소의 진입점(entrypoint)인 `run_fn` 함수가 포함된 Python 파일을 작성해야 합니다.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

이제 TFX 파이프라인 구축에 필요한 모든 준비 단계를 완료했습니다.

### 파이프라인 정의 작성하기

TFX 파이프라인을 생성하는 함수를 정의합니다. `Pipeline` 객체는 TFX가 지원하는 파이프라인 오케스트레이션 시스템 중 하나를 사용하여 실행할 수 있는 TFX 파이프라인을 나타냅니다.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## 파이프라인 실행하기

TFX는 파이프라인을 실행하는 여러 오케스트레이터를 지원합니다. 이 튜토리얼에서는 TFX Python 패키지에 포함되어 있고 로컬 환경에서 파이프라인을 실행하는 `LocalDagRunner`를 사용합니다. TFX 파이프라인은 종종 "DAG"라 불리기도 하며 이는 방향성 비순환 그래프(Directed Acyclic Graph)의 약자입니다.

`LocalDagRunner`는 개발 및 디버깅을 위한 빠른 반복 기능을 제공합니다. TFX는 프로덕션 사용 사례에 적합한 Kubeflow Pipelines 및 Apache Airflow 등의 다른 오케스트레이터도 지원합니다.

다른 오케스트레이션 시스템에 대해 자세히 알아보려면 [Cloud AI Platform 파이프라인의 TFX](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines) 혹은 [TFX Airflow 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop)을 참조하세요.

이제 `LocalDagRunner`를 생성하고 이미 정의한 함수로부터 생성한 `Pipeline` 객체를 전달합니다.

파이프라인은 직접 실행되며 ML 모델 훈련을 포함하여 파이프라인의 진행 상황에 대한 로그를 볼 수 있습니다.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

파이프라인이 성공적으로 완료되면 로그의 끝에 "INFO:absl:Component Pusher is finished." 메시지가 표시됩니다. 왜냐하면 `Pusher` 구성 요소가 파이프라인의 마지막 구성 요소이기 때문입니다.

이전 단계에서 변수를 변경하지 않은 경우 푸셔 구성 요소는 훈련한 모델을 `serving_model/penguin-simple` 디렉터리인 `SERVING_MODEL_DIR`로 푸시합니다. Colab의 왼쪽 패널에서 혹은 다음 명령을 사용하여 파일 브라우저에서 결과를 볼 수 있습니다.

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

## 다음 단계

https://www.tensorflow.org/tfx/tutorials에서 더 많은 리소스를 확인할 수 있습니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.
